# Text Features In CatBoost

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/catboost/tutorials/blob/master/events/2019_11_30_datastart.ipynb)

**Set GPU as hardware accelerator**

First of all, you need to select GPU as hardware accelerator. There are two simple steps to do so:
Step 1. Navigate to **Runtime** menu and select **Change runtime type**
Step 2. Choose **GPU** as hardware accelerator.
That's all!

Let's install CatBoost.

In [ ]:
!pip install catboost

In this tutorial we will use dataset **Rotten Tomatoes Movie Reviews** from [Kaggle](https://www.kaggle.com) competition for our experiments. Data can be downloaded [here](https://www.kaggle.com/rpnuser8182/rotten-tomatoes/data).

In [ ]:
import os
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)

import catboost
print(catboost.__version__)

## Reading the data

In [ ]:
from catboost.datasets import rotten_tomatoes

train_df, test_df = rotten_tomatoes()

train_df.head(2)

### Features description 

|Id | Feature name      |  Description                                                                                 |
|---|-------------------|----------------------------------------------------------------------------------------------|
| 1 | ``id``            |  unique movie id                                                                             |
| 2 | ``synopsis``      |  brief summary of the major points of a movie                                                |
| 3 | ``rating_MPAA``   |  film rating by MPAA rating system                                                           |
| 4 | ``genre``         |  list of genres that are suitable for this film (e.g. Action, Adventure, Comedy,...          |
| 5 | ``director``      |  list of persons who direct the making of a film                                             |
| 6 | ``writer``        |  list of persons who write a screenplay                                                      |
| 7 | ``theater_date``  |  the date when film was first shown to the public in cinema (string)                         |
| 8 | ``dvd_date``      |  the date when film was released on DVD (string)                                             |
| 9 | ``box_office``    |  the amount of money raised by ticket sales (revenue)                                        |
| 10 | ``runtime``      |  film duration in minutes                                                                    |
| 11 | ``studio``       |  is a major entertainment company or motion picture company (20th Century Fox, Sony Pictures)|
| 12 | ``dvd_date_int`` |  the date when film was released on DVD (converted to integer)                               |
| 13 | ``theater_date_int`` |  the date when film was first shown to the public in cinema (converted to integer)       |
| 14 | ``review``       |  review of a movie, that was written by a critic                                             |
| 15 | ``rating``       |  float rating from 0 to 1 of the film according to the Rotten tomatoes web site              |
| 16 | ``fresh``        |  freshness of review - fresh or rotten                                                       |
| 17 | ``critic``       |  name of reviewer                                                                            |
| 18 | ``top_critic``   |  binary feature, is reviewer a top critic or not                                             |
| 19 | ``publisher``    |  journal or website where the review was published                                           |
| 20 | ``date``         |  the date when critic publish review (string)                                                |
| 21 | ``date_int``     |  the date when critic publish review (converted to integer)                                  |
| 22 | ``rating_10``    |  integer rating from 0 to 10 of the film according to the critic                             |

We mark as **auxiliary** columnns 'id' and 'rating', because they can be the reason of overfitting, 'theater_date','dvd_date','date' because we convert them into integers.

We mark as **text** features 'synopsis' because it is short *text* description of a film, 'genre' because it is combination of categories (we know that strings have structure where words define categories), for example 'Action | Comedy | Adventure', 'director' and 'writer' features are included to the text features by the same reason, 'review' becuase it is a *text* summary of critic opinion.

We mark as **categorical** features 'rating_MPAA', 'studio', 'fresh', 'critic', 'top_critic' and 'publisher' because they can not be splitted into the group of categorical features and feature values can not be compared.

The other columns considered as **numeric**.

## Preparing the data

In [ ]:
auxiliary_columns = ['id', 'theater_date', 'dvd_date', 'rating', 'date']
cat_features = ['rating_MPAA', 'studio', 'fresh', 'critic', 'top_critic', 'publisher']
text_features = ['synopsis', 'genre', 'director', 'writer', 'review']

In [ ]:
def fill_na(df, features):
    for feature in features:
        df[feature].fillna('', inplace=True)

def preprocess_data_part(data_part):
    data_part = data_part.drop(auxiliary_columns, axis=1)

    fill_na(data_part, cat_features)
    fill_na(data_part, text_features)

    X = data_part.drop(['rating_10'], axis=1)
    y = data_part['rating_10']
    return X, y

X_train, y_train = preprocess_data_part(train_df)
X_test, y_test = preprocess_data_part(test_df)

X_train_no_text = X_train.drop(text_features, axis=1)
X_test_no_text = X_test.drop(text_features, axis=1)

In [ ]:
X_train_no_text.head(2)

In [ ]:
from catboost import Pool

train_pool_no_text = Pool(
    X_train_no_text, y_train, 
    cat_features=cat_features, 
)

validation_pool_no_text = Pool(
    X_test_no_text, y_test, 
    cat_features=cat_features, 
)

print('Train dataset shape: {}\n'.format(train_pool_no_text.shape))

## Training

In [ ]:
from catboost import CatBoostClassifier

simple_model = CatBoostClassifier(
    iterations=5,
    learning_rate=0.1,
)

simple_model.fit(
    train_pool_no_text,
    eval_set=validation_pool_no_text,
    verbose=False
)

print('Model is fitted: {}'.format(simple_model.is_fitted()))
print('Model params:\n{}'.format(simple_model.get_params()))

## Stdout of the training

In [ ]:
simple_model = CatBoostClassifier(
    iterations=15,
    # verbose=5,
)
simple_model.fit(train_pool_no_text, eval_set=validation_pool_no_text);

In [ ]:
model_no_text = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    eval_metric='Accuracy',
    task_type='GPU',
)

model_no_text.fit(
    train_pool_no_text,
    eval_set=validation_pool_no_text,
    verbose=100,
);

## Model predictions

In [ ]:
preds = model_no_text.predict_proba(validation_pool_no_text)

print(preds)
print(preds.shape)

In [ ]:
print(model_no_text.predict(validation_pool_no_text))

# Text Features

In [ ]:
def fit_model(X_train, y_train, X_test, y_test, **kwargs):
    train_pool = Pool(
        X_train, y_train, 
        cat_features=cat_features,
        text_features=text_features,
    )

    validation_pool = Pool(
        X_test, y_test, 
        cat_features=cat_features,
        text_features=text_features,
    )

    model = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.05,
        eval_metric='Accuracy',
        task_type='GPU',
        **kwargs
    )

    return model.fit(
        train_pool,
        eval_set=validation_pool,
        verbose=100,
    )

In [ ]:
model = fit_model(X_train, y_train, X_test, y_test)

In [ ]:
def print_score_diff(first_model, second_model):
    first_accuracy = first_model.best_score_['validation']['Accuracy']
    second_accuracy = second_model.best_score_['validation']['Accuracy']

    gap = (second_accuracy - first_accuracy) / first_accuracy * 100

    print('{} vs {} ({:+.2f}%)'.format(first_accuracy, second_accuracy, gap))

In [ ]:
print_score_diff(model_no_text, model)

<span style="color:red">Note!</span>

1. Text features also cannot contain NaN values, so we converted them into strings manually.
2. Training with text features supported only on GPU.
3. The training may be performed only with classification losses and targets.

## How it works?

1. **Text Tokenization**
2. **Dictionary Creation**
3. **Feature Calculation**

## Text Tokenization

Usually we get our text as a sequence of Unicode symbols. So, if the task isn't a DNA classification we don't need such granularity, moreover, we need to extract more complicated entities, e.g. words. The process of extraction tokens -- words, numbers, punctuation symbols or special symbols which defines emoji from a sequence is called **tokenization**.<br>

Tokenization is the first part of text preprocessing in CatBoost and performed as a simple splitting a sequence on a string pattern (e.g. space).

In [ ]:
text_small = [
    "Cats are so cute :)",
    "Mouse scare...",
    "The cat defeated the mouse",
    "Cute: Mice gather an army!",
    "Army of mice defeated the cat :(",
    "Cat offers peace",
    "Cat is scared :(",
    "Cat and mouse live in peace :)"
]

target_small = [1, 0, 1, 1, 0, 1, 0, 1]

In [ ]:
from catboost.text_processing import Tokenizer

simple_tokenizer = Tokenizer()

def tokenize_texts(texts):
    return [simple_tokenizer.tokenize(text) for text in texts]

simple_tokenized_text = tokenize_texts(text_small)
simple_tokenized_text

### More preprocessing!

Lets take a closer look on the tokenization result of small text example -- the tokens contains a lot of mistakes:

1. They are glued with punctuation 'Cute:', 'army!', 'skare...'.
2. The words 'Cat' and 'cat', 'Mice' and 'mice' seems to have same meaning, perhaps they should be the same tokens.
3. The same problem with tokens 'are'/'is' -- they are inflected forms of same token 'be'.

**Punctuation handling**, **lowercasing**, and **lemmatization** processes help to overcome these problems.

### Punctuation handling and lowercasing

In [ ]:
tokenizer = Tokenizer(
    lowercasing=True,
    separator_type='BySense',
    token_types=['Word', 'Number']
)

tokenized_text = [tokenizer.tokenize(text) for text in text_small]
tokenized_text

### Removing stop words

**Stop words** - the words that are considered to be uninformative in this task, e.g. function words such as *the, is, at, which, on*.
Usually stop words are removed during text preprocessing to reduce the amount of information that is considered for further algorithms.
Stop words are collected manually (in dictionary form) or automatically, for example taking the most frequent words.

In [ ]:
stop_words = set(('be', 'is', 'are', 'the', 'an', 'of', 'and', 'in'))

def filter_stop_words(tokens):
    return list(filter(lambda x: x not in stop_words, tokens))
    
tokenized_text_no_stop = [filter_stop_words(tokens) for tokens in tokenized_text]
tokenized_text_no_stop

### Lemmatization

Lemma (Wikipedia) -- is the canonical form, dictionary form, or citation form of a set of words.<br>
For example, the lemma "go" represents the inflected forms "go", "goes", "going", "went", and "gone".<br>
The process of convertation word to its lemma called **lemmatization**.


In [ ]:
import nltk

nltk_data_path = os.path.join(os.path.dirname(nltk.__file__), 'nltk_data')
nltk.data.path.append(nltk_data_path)
nltk.download('wordnet', nltk_data_path)

lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_tokens_nltk(tokens):
    return list(map(lambda t: lemmatizer.lemmatize(t), tokens))

In [ ]:
text_small_lemmatized_nltk = [lemmatize_tokens_nltk(tokens) for tokens in tokenized_text_no_stop]
text_small_lemmatized_nltk

Now words with same meaning represented by the same token, tokens are not glued with punctuation.

<span style="color:red">Be carefull.</span> You should verify for your own task:<br>
Is it realy necessary to remove punctuation, lowercasing sentences or performing a lemmatization and/or by word tokenization?<br>

### Let's check up accuracy with new text preprocessing

Since CatBoost doesn't perform spacing punctuation, lowercasing letters and lemmatization, we need to preprocess text manually and then pass it to learning algorithm.

Since the natural text features is only synopsis and review, we will preprocess only them.

In [ ]:
%%time

def preprocess_data(X):
    X_preprocessed = X.copy()
    for feature in ['synopsis', 'review']:
        X_preprocessed[feature] = X[feature].apply(lambda x: ' '.join(lemmatize_tokens_nltk(tokenizer.tokenize(x))))
    return X_preprocessed

X_preprocessed_train = preprocess_data(X_train)
X_preprocessed_test = preprocess_data(X_test)

In [ ]:
model_on_processed_data = fit_model(X_preprocessed_train, y_train, X_preprocessed_test, y_test)

In [ ]:
print_score_diff(model, model_on_processed_data)

## Dictionary Creation

After the first stage, preprocessing of text and tokenization, the second stage starts. The second stage uses the prepared text to select a set of units, which will be used for building new numerical features.

A set of selected units is called dictionary. It might contain words, word bigramms, or character n-gramms.

In [ ]:
from catboost.text_processing import Dictionary

In [ ]:
dictionary = Dictionary(occurence_lower_bound=0, max_dictionary_size=10)

dictionary.fit(text_small_lemmatized_nltk);
#dictionary.fit(text_small, tokenizer)

In [ ]:
dictionary.save('dictionary.tsv')
!cat dictionary.tsv

## Feature Calculation

### Convertation into fixed size vectors

The majority of classic ML algorithms are computing and performing predictions on a fixed number of features $F$.<br>
That means that learning set $X = \{x_i\}$ contains vectors $x_i = (a_0, a_1, ..., a_F)$ where $F$ is constant.

Since text object $x$ is not a fixed length vector, we need to perform preprocessing of the origin set $D$.<br>
One of the simplest text to vector encoding technique is **Bag of words (BoW)**.

### Bag of words algorithm

The algorithm takes in a dictionary and a text.<br>
During the algorithm text $x = (a_0, a_1, ..., a_k)$ converted into vector $\tilde x = (b_0, b_1, ..., b_F)$,<br> where $b_i$ is number of occurences word with id=$i$ from dictionary into text $x$.

In [ ]:
text_small_lemmatized_nltk

In [ ]:
dictionary.apply([text_small_lemmatized_nltk[0]])

In [ ]:
def bag_of_words(tokenized_text, dictionary):
    features = np.zeros((len(tokenized_text), dictionary.size))
    for i, tokenized_sentence in enumerate(tokenized_text):
        indices = np.array(dictionary.apply([tokenized_sentence])[0])
        features[i, indices] = 1
    return features

bow_features = bag_of_words(text_small_lemmatized_nltk, dictionary)
bow_features

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import csr_matrix
from sklearn.metrics import log_loss

def fit_linear_model(X, c):
    model = LogisticRegression()
    model.fit(X, c)
    return model

def fit_naive_bayes(X, c):
    clf = MultinomialNB()
    if isinstance(X, csr_matrix):
        X.eliminate_zeros()
    clf.fit(X, c)
    return clf

def evaluate_model_logloss(model, X, y):
    y_pred = model.predict_proba(X)[:,1]
    metric = log_loss(y, y_pred)
    print('Logloss: ' + str(metric))

In [ ]:
def evaluate_models(X, y):
    linear_model = fit_linear_model(bow_features, target_small)
    naive_bayes = fit_naive_bayes(bow_features, target_small)
        
    print('Linear model')
    evaluate_model_logloss(linear_model, X, y)
    print('Naive bayes')
    evaluate_model_logloss(naive_bayes, X, y)
    print('Comparing to constant prediction')
    logloss_constant_prediction = log_loss(y, np.ones(shape=(len(text_small), 2)) * 0.5)
    print('Logloss: ' + str(logloss_constant_prediction))
    
evaluate_models(bow_features, target_small)

In [ ]:
dictionary = Dictionary(occurence_lower_bound=0)
dictionary.fit(text_small_lemmatized_nltk)

bow_features = bag_of_words(text_small_lemmatized_nltk, dictionary)
evaluate_models(bow_features, target_small)

### Looking at sequences of letters / words

Let's look at the example: texts 'The cat defeated the mouse' and 'Army of mice defeated the cat :('<br>
Simplifying it we have three tokens in each sentence 'cat defeat mouse' and 'mouse defeat cat'.<br>
After applying BoW we get two equal vectors with the opposite meaning:

| cat | mouse | defeat |
|-----|-------|--------|
| 1   | 1     | 1      |
| 1   | 1     | 1      |

How to distinguish them?
Lets add sequences of words as a single tokens into our dictionary:

| cat | mouse | defeat | cat_defeat | mouse_defeat | defeat_cat | defeat_mouse |
|-----|-------|--------|------------|--------------|------------|--------------|
| 1   | 1     | 1      | 1          | 0            | 0          | 1            |
| 1   | 1     | 1      | 0          | 1            | 1          | 0            |

**N-gram** is a continguous sequence of $n$ items from a given sample of text or speech (Wikipedia).<br>
In example above Bi-gram (Bigram) = 2-gram of words.

Ngrams help to add into vectors more information about text structure, moreover there are n-grams has no meanings in separation, for example, 'Mickey Mouse company'.

In [ ]:
dictionary = Dictionary(occurence_lower_bound=0, gram_order=2)
dictionary.fit(text_small_lemmatized_nltk)

dictionary.save('dictionary.tsv')
!cat dictionary.tsv

In [ ]:
bow_features = bag_of_words(text_small_lemmatized_nltk, dictionary)
evaluate_models(bow_features, target_small)

### Unigram + Bigram

In [ ]:
dictionary1 = Dictionary(occurence_lower_bound=0)
dictionary1.fit(text_small_lemmatized_nltk)

bow_features1 = bag_of_words(text_small_lemmatized_nltk, dictionary1)

dictionary2 = Dictionary(occurence_lower_bound=0, gram_order=2)
dictionary2.fit(text_small_lemmatized_nltk)

bow_features2 = bag_of_words(text_small_lemmatized_nltk, dictionary2)

bow_features = np.concatenate((bow_features1, bow_features2), axis=1)
evaluate_models(bow_features, target_small)

## CatBoost Configuration

Parameter names:

1. **Text Tokenization** - No Parameter
2. **Dictionary Creation** - `dictionaries`
3. **Feature Calculation** - `text_processing`

### `dictionaries`

Dictionaries parameters specified as list of strings, each string is a description of dictionary in the following format:<br>
``'DictionaryName:[Param1=Value1,[Param2=Value2]]'``

**Parameters**:

``min_token_occurrence`` -- number; minimal token occurence to enter to dictionary<br>
This parameter can be very useful for filtering too rare tokens, this helps to avoid overfitting.

``max_dict_size`` -- number; maximum dictionary size<br>
This parameter can help to control the size of model.

``token_level_type`` -- string: ``Word`` or ``Letter``; use letter or word tokens

``gram_order`` -- number; build n-gram dictionary.

**Examples**:
1. `'Word:max_dict_size=50000'`<br>
This means unigram dictionary of size 50000 will be build.

2. `'BiGram:gram_order=2'`<br>
This means bigram dictionary will be build.

### `text_processing`

Text processing parameters specified as a list of strings, each string is a description of feature preprocessing in the following format:<br>
`'[FeatureId~][FeatureEstimator1+DictionaryName1[|FeatureEstimator2+DictionaryName2]]'`

**Feature Estimators**:

1. **`BoW`**<br>
Bag of words: 0/1 features (text sample has or not token_id).<br>
Number of produced numeric features = dictionary size.<br>
Parameters: `top_tokens_count` - maximum number of tokens that will be used for vectorization in bag of words, the most frequent $n$ tokens are taken (**highly affect both on CPU ang GPU RAM usage**).

2. **`NaiveBayes`**<br>
NaiveBayes: [Multinomial naive bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Multinomial_naive_Bayes) model, produce number of features equal to number of classes.<br> To avoid target leakage this model is computed online on a several dataset permutations like we do with [estimation of CTRs](https://catboost.ai/docs/concepts/algorithm-main-stages_cat-to-numberic.html).

3. **`BM25`**<br>
[BM25](https://en.wikipedia.org/wiki/Okapi_BM25) also computed online, it's a function that used for ranking purposes by search engines to estimate the relevance of documents.

**Examples**:
1. `'BoW+Word|NaiveBayes+Bigram'`<br>
This means that for all text features `BoW` and `NaiveBayes` features will be computed using `Word` and ``Bigram`` dictionaries correspondingly.

2. `'0~BoW+Word|NaiveBayes+Word,Bigram'`<br>
This means that on 0-th text feature `BoW` and `NaiveBayes` features will be computed using ``Word`` and `Word,Bigram` dictionaries correspondingly.<br>

Dictionaries names are taken from `dictionaries` parameter.

In [ ]:
model_on_processed_data_2 = fit_model(
    X_preprocessed_train, y_train,
    X_preprocessed_test, y_test,
    dictionaries = [
        'Word:min_token_occurrence=5,max_dict_size=50000',
        'BiGram:gram_order=2,min_token_occurrence=5,max_dict_size=50000'
    ],
    text_processing = [
        'NaiveBayes+Word|BoW+Word,BiGram|BM25+Word'
    ],
)

In [ ]:
print_score_diff(model_no_text, model_on_processed_data)

# Summary: Text features in CatBoost

### The algorithm:
1. Input text is loaded as a usual column. ``text_column: [string]``.
2. Each text sample is tokenized via splitting by space. ``tokenized_column: [[string]]``.
3. Dictionary estimation.
4. Each string in tokenized column is converted into token_id from dictionary. ``text: [[token_id]]``.
5. Depending on the parameters CatBoost produce features basing on the resulting text column: Bag of words, Multinomial naive bayes or Bm25.
6. Computed float features are passed into the usual CatBoost learning algorithm.